In [1]:
import seaborn as sns
import numpy as np
import nibabel as nib
from nilearn import datasets
from statsmodels.stats.multitest import multipletests
from src.custom_plotting import plot_surface_stats, mkNifti
from nilearn import surface
from pathlib import Path

import matplotlib.pyplot as plt
from matplotlib import gridspec
import itertools
from nilearn.plotting import plot_surf_roi

In [2]:
def load_noise_ceiling(mask_dir, sid, mask, dataset):
    noise_ceiling = np.load(f'{mask_dir}/sub-{sid}_set-{dataset}_stat-rho_statmap.npy')
    noise_ceiling = noise_ceiling[mask]
    noise_ceiling[noise_ceiling <= 0.] = np.nan
    return noise_ceiling

In [3]:
def filter_r(rs, ps, p_crit=0.05, correct=False, threshold=False):
    if correct and (ps is not None):
        _, ps_corrected, _, _ = multipletests(ps, method='fdr_bh')
    elif not correct and (ps is not None):
        ps_corrected = ps.copy()
    else:
        ps_corrected = None

    if threshold and (ps is not None):
        rs[ps_corrected >= p_crit] = 0.
    else:
        rs[rs < 0.] = 0.
    return rs, ps_corrected

In [4]:
def plot(r_, p_, sid, mask, img, fsaverage, method, cmap):
    r_, _ = filter_r(r_, p_)
    if method == 'test':
        dataset = 'test'
    else:
        dataset = 'train'
    volume = mkNifti(r_, mask, img)
    texture = {'left': surface.vol_to_surf(volume, fsaverage['pial_left']),
               'right': surface.vol_to_surf(volume, fsaverage['pial_right'])}

    # plotting.plot_glass_brain(volume,
    #                           threshold=1e-2, vmax=1.,
    #                           colorbar=True, symmetric_cbar=False,
    #                           cmap=cmap,
    #                           output_file=f'{figure_dir}/sub-{sid}_volume.pdf')

    vmax = 0.1
    threshold = np.nanmin(r_[np.invert(np.isclose(r_, 0.))])
    print(threshold)
    plot_surface_stats(fsaverage, texture,
                       roi=roi_parcel,
                       cmap=cmap,
                       modes=['lateral', 'ventral'],
                       threshold=threshold,
                       vmax=vmax,
                       cbar_title=r"Correlation ($r$)",
                       output_file=f'{figure_dir}/sub-{sid}.pdf')

In [5]:
def mk_cmap():
    palette = {'black': (0., 0., 0.),
               'gray': (0.25, 0.25, 0.25),
               'mustard': (0.95703125, 0.86328125, 0.25),
               'purple': (0.51953125, 0.34375, 0.953125),
               'cyan': (0.44921875, 0.8203125, 0.87109375),
               'reddish': (0.8515625, 0.32421875, 0.35546875)}
    pref_cmap = sns.color_palette('Paired', len(palette.keys()), as_cmap=True)
    pref_cmap._lut = pref_cmap._lut[:9]

    out_colors = []
    for i, feature in enumerate(palette.keys()):
        print(feature)
        rgb = palette[feature]
        out_colors.append(rgb)
        pref_cmap._lut[i] = list(rgb) + [1.]
    pref_cmap.colors = tuple(out_colors)

    pref_cmap._lut[6] = [0., 0., 0., 1.]
    pref_cmap._lut[7] = [0., 0., 0., 1.]
    pref_cmap._lut[8] = [0., 0., 0., 1.]

    pref_cmap.N = 6
    pref_cmap._i_over = 6
    pref_cmap._i_under = 7
    pref_cmap._i_bad = 8
    return pref_cmap

In [6]:
def plot_pref_map(texture, cmap, output_file=None):
    hemis = ['left', 'right']
    modes = ['lateral', 'ventral', 'medial']
    threshold = 1.

    cbar_h = .25
    title_h = 0
    w, h = plt.figaspect((len(modes) + cbar_h + title_h) / len(hemis)) * 2
    fig = plt.figure(figsize=(w, h), constrained_layout=False)
    height_ratios = [title_h] + [1.] * len(modes) + [cbar_h]
    grid = gridspec.GridSpec(
        len(modes) + 2, len(hemis),
        left=0., right=1., bottom=0., top=1.,
        height_ratios=height_ratios, hspace=0.0, wspace=0.0)
    axes = []
    for i, (mode, hemi) in enumerate(itertools.product(modes, hemis)):
        bg_map = fsaverage['sulc_%s' % hemi]

        ax = fig.add_subplot(grid[i + len(hemis)], projection="3d")
        axes.append(ax)
        plot_surf_roi(surf_mesh=fsaverage[f'infl_{hemi}'],
                      roi_map=texture[hemi],
                      view=mode, hemi=hemi,
                      bg_map=bg_map,
                      alpha=1.,
                      axes=ax,
                      colorbar=False,  # Colorbar created externally.
                      # vmax=vmax,
                      threshold=threshold,
                      cmap=cmap)

        rect = ax.patch
        rect.set_facecolor('white')

    if output_file is not None:
        fig.savefig(output_file, facecolor='white')
    plt.close(fig)

In [7]:
data_dir='/Users/emcmaho7/Dropbox/projects/SI_fmri/SIfMRI_analysis/data/raw'
out_dir = '/Users/emcmaho7/Dropbox/projects/SI_fmri/SIfMRI_analysis/data/interim'
mask_dir = '/Users/emcmaho7/Dropbox/projects/SI_fmri/SIfMRI_analysis/data/interim/Reliability'
top_fig_dir = '/Users/emcmaho7/Dropbox/projects/SI_fmri/SIfMRI_analysis/reports/figures/PlotVoxelEncoding'

fsaverage = datasets.fetch_surf_fsaverage()
cmap_pred = sns.color_palette('magma', as_cmap=True)
pref_cmap = mk_cmap()
roi_parcel = None
img = nib.load(f'{mask_dir}/sub-01_set-test_stat-rho_statmap.nii.gz')
n_perm = 5000
n_subjs = 4
method = 'CV'

black
gray
mustard
purple
cyan
reddish


In [8]:
def full_model_mask(out_dir, sid, method):
    r = np.load(f'{out_dir}/VoxelPermutation/sub-{sid}_prediction-all_method-{method}_rs.npy')
    p = np.load(f'{out_dir}/VoxelPermutation/sub-{sid}_prediction-all_method-{method}_ps.npy')
    _, ps_corrected = filter_r(r, p)
    return ps_corrected >= 0.05

In [9]:
def compute_pref_map(arr):
    max_count = np.sum(np.isclose(arr, arr.max(axis=0)), axis=0)
    out = arr.argmax(axis=0)
    out[max_count > 1] = 0
    return out

In [10]:
# models = ['affective', 'social', 'primitive', 'visual', 'nuissance']
# models.reverse()
models = ['facingness', 'joint action', 'communication']

In [11]:
for sid in ['01', '02', '03', '04']:
    groups = None
    for model in models:
        # print(f'sub-{sid}: {model}')
        figure_dir = f'{top_fig_dir}/{method}/wo_valence_arousal/{model}'
        Path(figure_dir).mkdir(parents=True, exist_ok=True)

        r_full = np.load(f'{out_dir}/VoxelPermutation/sub-{sid}_prediction-all_drop-None_method-{method}_rs.npy')
        r_dropped = np.load(f'{out_dir}/VoxelPermutation/sub-{sid}_prediction-all_drop-{model}_method-{method}_rs.npy')
        r = r_full**2 - r_dropped**2
        # p = np.load(f'{out_dir}/VoxelPermutation/sub-{sid}_prediction-{model}_method-{method}_ps.npy')
        mask = np.load(f'{out_dir}/Reliability/sub-{sid}_set-test_reliability-mask.npy').astype('bool')
        # if groups is None:
        #     groups = []
        #     groups.append(np.zeros_like(r))
        # groups.append(r)
        plot(r, None, sid, mask, img, fsaverage, method, cmap_pred)

    # figure_dir = f'{top_fig_dir}/{method}/full/pref_map'
    # Path(figure_dir).mkdir(parents=True, exist_ok=True)
    # pref_map = compute_pref_map(np.vstack(groups))
    # insig_voxels = full_model_mask(out_dir, sid, method)
    # pref_map[insig_voxels] = 0.
    # volume = mkNifti(pref_map, mask, img)
    # texture = {'left': np.round(surface.vol_to_surf(volume, fsaverage['pial_left'],
    #                                                 interpolation='nearest')),
    #            'right': np.round(surface.vol_to_surf(volume, fsaverage['pial_right'],
    #                                                  interpolation='nearest'))}
    # plot_pref_map(pref_cmap, f'{figure_dir}/sub-{sid}.pdf')

2.025755024663567e-08
8.850072707303613e-08
2.3463626501831793e-07
3.3220101102267983e-08
2.3713171045001914e-08
4.8696334889867875e-08
1.5517726860916485e-08
5.473188734866907e-08
2.005029582688694e-08
1.3111973664073736e-08
3.003443051923828e-08
2.3100724250157967e-08
